In [ ]:
!pip3 install optuna==0.14.0
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 91 kB 3.9 MB/s 
     |████████████████████████████████| 81 kB 8.0 MB/s 
     |████████████████████████████████| 209 kB 55.7 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 147 kB 65.9 MB/s 
     |████████████████████████████████| 112 kB 67.1 MB/s 
  Created wheel for optuna: filename=optuna-0.14.0-py3-none-any.whl size=125709 sha256=5b2309556e9edd9a3255913f8c2898fad8e7e79746d2c581437bb244b3733df4
  Stored in directory: /root/.cache/pip/wheels/c1/41/64/03b183676c5d5e978de160cab6268d5b4fb095dff63f720e01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=a609ece9b60e063080931dc671f8139217858f40fcef9d7df71c49e1af3e9648
  Stored in directory: /root/.ca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 60.9 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf

from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Activation, Flatten, Dense, Conv2D, Conv1D,Input
from keras.layers import MaxPooling1D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from keras.layers.core import *
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import keras.backend as K
import optuna
import math
import random
from copy import deepcopy

In [ ]:
print('Keras', keras.__version__)
print('TensorFlow', tf.__version__)
# import Optuna and OptKeras after Keras
print('Optuna', optuna.__version__)

Keras 2.8.0
TensorFlow 2.8.2
Optuna 0.14.0


In [ ]:
train_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_train_main.parquet.snappy",engine='fastparquet')
#subsample for nest CV
np.random.seed(760)
train_df=train_df.loc[np.random.choice(train_df.index, 80000, replace=False)].reset_index()
test_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_test_main.parquet.snappy",engine='fastparquet')
test_df=test_df.loc[np.random.choice(test_df.index, 20000, replace=False)].reset_index()

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

training_withaim=train_df.drop(labels="r_id", axis=1)
testing_withaim=test_df.drop(labels="r_id", axis=1)

#Check the NaN in data and drop them
imp_train=SimpleImputer(missing_values=np.NaN)
training=pd.DataFrame(imp_train.fit_transform(training_withaim))

imp_test=SimpleImputer(missing_values=np.NaN)
testing=pd.DataFrame(imp_test.fit_transform(testing_withaim))
print(training)
# There aren't any nan data in the dataframe 
training.isnull().values.sum()
testing.isnull().values.sum()
training = training.iloc[: , 0:18]
testing = testing.iloc[: , 0:18]
scaler = StandardScaler()

# scale skewed and target features
std_train_df = train_df.copy(deep=True)
std_train_df = scaler.fit_transform(training)
std_test_df = test_df.copy(deep=True)
std_test_df = scaler.transform(testing)
#std_test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])

print(std_train_df[0])
std_train_df = pd.DataFrame(std_train_df)
std_test_df = pd.DataFrame(std_test_df)

training = std_train_df.iloc[: , 0:18]
testing = std_test_df.iloc[: , 0:18]
labelsForTrain=training_withaim.iloc[: , -1]
labelsForTest=testing_withaim.iloc[: , -1]
input_shape = training.shape

             0    1     2      3      4      5          6         7    8   \
0      268123.0  5.0  25.0  133.0  418.0  630.0  72.978978  0.400000  8.0   
1      340360.0  1.0   1.0  425.0   93.0   17.0  57.201016  0.000000  0.0   
2      245244.0  4.0  16.0   78.0  375.0  506.0  63.609630  1.705534  8.0   
3      135424.0  4.0  16.0  114.0   66.0  196.0  76.291694  0.535714  3.0   
4      267072.0  1.0   1.0  225.0   16.0    1.0  29.199427  0.000000  0.0   
...         ...  ...   ...    ...    ...    ...        ...       ...  ...   
79995  214952.0  5.0  25.0   46.0    4.0   95.0  64.888960  0.884211  1.0   
79996  181917.0  1.0   1.0   30.0  243.0    6.0   0.000004  0.000000  0.0   
79997  185008.0  5.0  25.0  170.0  188.0  436.0  58.579512  0.924312  7.0   
79998  247910.0  2.0   4.0  280.0  103.0  137.0  16.853052  0.613139  2.0   
79999   84136.0  2.0   4.0  116.0    1.0    9.0  24.637658  0.000000  0.0   

         9     10      11   12      13   14     15        16        17     

In [ ]:
training.shape

(80000, 18)

In [ ]:
def create_model(activation, num_hidden_layer, num_hidden_unit):
  inputs = Input(shape=(training.shape[1],))
  model = inputs
  for i in range(1,num_hidden_layer):
    model = Dense(num_hidden_unit, activation=activation,)(model)
        
        
  model = Dense(1,)(model)
  model = Model(inputs, model)

  return model

## Code for nested CV

In [ ]:
from sklearn.utils import validation
def objective(trial:optuna.Trial,data_train,result_train,data_val,result_val):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  #Leave fewer optimizer
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',2,4)
  #get more features per layer, add num of hidden unit if have time

  #define the number of unit with 2^n
  i = trial.suggest_int('i',3,9)
  num_hidden_unit = 2**i
  
  #Try to adjust learning_rate

  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001,0.01)
  
  # Gradient Clipping

  optimizer = Adam(learning_rate=learning_rate,clipnorm=1.0)
    
  num_folds = 3

  loss_per_fold = []
  es = EarlyStopping(monitor='val_mse', patience=5)

  model = create_model(activation,num_hidden_layer,num_hidden_unit)
  model_list.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  #learning scheduler
  reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=5, min_lr=0.001)
      # Fit data to model
  score_int=[]
  history = model.fit(data_train, result_train,
                  batch_size=64,
                  epochs=500,
                  verbose=2,
                  validation_data=(data_val,result_val),
                  validation_batch_size=64,
                  callbacks=[es,reduce_lr])
  
  scores=model.evaluate(data_val,result_val,verbose=0)

  print(f'Score for inner fold : {model.metrics_names[0]} of {scores[0]}')
  score_int.append(round(scores[1],2))
  #loss_per_fold.append(scores[0])

  history_list.append(history)
  return np.mean(score_int)

In [ ]:
def objective_cv(trial:optuna.Trial,train_outer,result_outer)->np.array:
  kfold = KFold(n_splits=3,shuffle=True)
  scores=list()
  print(f'Trial Number:{trial.number}')
  for fold, (train,test) in enumerate(kfold.split(train_outer,result_outer)):
    print(f"-----Fold:{fold}--------")

    new_train_inner=train_outer.iloc[train]
    new_label=result_outer.iloc[train]

    val_inner=train_outer.iloc[test]
    result_inner=train_outer.iloc[test]
    
    val_loss = objective(trial, new_train_inner,new_label,val_inner,result_inner)
    scores.append(val_loss)
  return np.mean(scores)

In [ ]:
def nested_cross_val(training,labelsForTrain):
  kfold = KFold(n_splits=3,shuffle=True)
  scores=list()
  for fold, (train_idx, val_idx) in enumerate(kfold.split(training,labelsForTrain)):
      print(f"------ Outer Fold - {fold} -------- ")
      train_outer=training.iloc[train_idx]
      result_outer=labelsForTrain.iloc[train_idx]
      study = optuna.create_study(
            study_name = f"NN-study-outer-fold{fold}",
            direction = "minimize",
            load_if_exists=True
        )
      func = lambda trial: objective_cv(trial, train_outer,result_outer)
      study.optimize(
            func,
            n_trials=50,
        )
      print(f"Number of finished trial : {len(study.trials)}")
      print(f"Best Trial Value : {study.best_trial.value}")

In [ ]:
model_list=[]
history_list=[]
nested_cross_val(training,labelsForTrain)

------ Outer Fold - 0 -------- 
Trial Number:0
-----Fold:0--------
Epoch 1/500
556/556 - 2s - loss: 34.0137 - mse: 34.0137 - mae: 1.6310 - val_loss: 13.7315 - val_mse: 13.7315 - val_mae: 2.7998 - lr: 0.0026 - 2s/epoch - 3ms/step
Epoch 2/500
556/556 - 1s - loss: 33.1798 - mse: 33.1798 - mae: 1.5914 - val_loss: 13.7150 - val_mse: 13.7150 - val_mae: 2.7937 - lr: 0.0026 - 1s/epoch - 2ms/step
Epoch 3/500
556/556 - 1s - loss: 32.3004 - mse: 32.3004 - mae: 1.5690 - val_loss: 13.2640 - val_mse: 13.2640 - val_mae: 2.7402 - lr: 0.0026 - 1s/epoch - 2ms/step
Epoch 4/500
556/556 - 1s - loss: 32.1878 - mse: 32.1878 - mae: 1.5625 - val_loss: 14.6481 - val_mse: 14.6481 - val_mae: 2.8382 - lr: 0.0026 - 1s/epoch - 2ms/step
Epoch 5/500
556/556 - 1s - loss: 32.7200 - mse: 32.7200 - mae: 1.5599 - val_loss: 14.1608 - val_mse: 14.1608 - val_mae: 2.8368 - lr: 0.0026 - 1s/epoch - 2ms/step
Epoch 6/500
556/556 - 1s - loss: 32.6926 - mse: 32.6926 - mae: 1.5587 - val_loss: 12.2293 - val_mse: 12.2293 - val_mae: 2.6

[I 2022-10-10 09:12:19,164] Finished trial#0 resulted in value: 14.479999999999999. Current best value is 14.479999999999999 with parameters: {'activation': 'relu', 'num_hidden_layer': 2, 'i': 7, 'learning_rate': 0.0026033795314011196}.


Trial Number:1
-----Fold:0--------
Epoch 1/500
556/556 - 2s - loss: 36.3448 - mse: 36.3448 - mae: 1.8240 - val_loss: 8.7354 - val_mse: 8.7354 - val_mae: 2.5830 - lr: 0.0010 - 2s/epoch - 3ms/step
Epoch 2/500
556/556 - 1s - loss: 33.2038 - mse: 33.2038 - mae: 1.6028 - val_loss: 9.1771 - val_mse: 9.1771 - val_mae: 2.5921 - lr: 0.0010 - 1s/epoch - 2ms/step
Epoch 3/500
556/556 - 1s - loss: 32.6691 - mse: 32.6691 - mae: 1.5754 - val_loss: 9.7745 - val_mse: 9.7745 - val_mae: 2.5916 - lr: 0.0010 - 1s/epoch - 2ms/step
Epoch 4/500
556/556 - 1s - loss: 32.3196 - mse: 32.3196 - mae: 1.5593 - val_loss: 9.8126 - val_mse: 9.8126 - val_mae: 2.6036 - lr: 0.0010 - 1s/epoch - 2ms/step
Epoch 5/500
556/556 - 1s - loss: 32.0748 - mse: 32.0748 - mae: 1.5471 - val_loss: 10.6782 - val_mse: 10.6782 - val_mae: 2.6823 - lr: 0.0010 - 1s/epoch - 2ms/step
Epoch 6/500
556/556 - 1s - loss: 31.8685 - mse: 31.8685 - mae: 1.5456 - val_loss: 10.3877 - val_mse: 10.3877 - val_mae: 2.6785 - lr: 0.0010 - 1s/epoch - 2ms/step
S

[I 2022-10-10 09:12:43,013] Finished trial#1 resulted in value: 10.4. Current best value is 10.4 with parameters: {'activation': 'tanh', 'num_hidden_layer': 2, 'i': 5, 'learning_rate': 0.001033313287653061}.


Trial Number:2
-----Fold:0--------
Epoch 1/500
556/556 - 2s - loss: 11.9834 - mse: 11.9834 - mae: 1.6250 - val_loss: 15.5860 - val_mse: 15.5860 - val_mae: 2.7325 - lr: 0.0015 - 2s/epoch - 3ms/step
Epoch 2/500
556/556 - 1s - loss: 11.3987 - mse: 11.3987 - mae: 1.5740 - val_loss: 16.0762 - val_mse: 16.0762 - val_mae: 2.7411 - lr: 0.0015 - 1s/epoch - 2ms/step
Epoch 3/500
556/556 - 1s - loss: 11.3440 - mse: 11.3440 - mae: 1.5721 - val_loss: 14.7112 - val_mse: 14.7112 - val_mae: 2.6418 - lr: 0.0015 - 1s/epoch - 2ms/step
Epoch 4/500
556/556 - 1s - loss: 11.2318 - mse: 11.2318 - mae: 1.5692 - val_loss: 15.3170 - val_mse: 15.3170 - val_mae: 2.7651 - lr: 0.0015 - 1s/epoch - 2ms/step
Epoch 5/500
556/556 - 1s - loss: 11.3911 - mse: 11.3911 - mae: 1.5671 - val_loss: 13.4293 - val_mse: 13.4293 - val_mae: 2.6655 - lr: 0.0015 - 1s/epoch - 2ms/step
Epoch 6/500
556/556 - 1s - loss: 11.4289 - mse: 11.4289 - mae: 1.5750 - val_loss: 13.3429 - val_mse: 13.3429 - val_mae: 2.6051 - lr: 0.0015 - 1s/epoch - 2m

...

Score for inner fold : loss of 10.22641658782959
Trial Number:48
-----Fold:0--------
Epoch 1/500
556/556 - 2s - loss: 23.1271 - mse: 23.1271 - mae: 2.5215 - val_loss: 1.2553 - val_mse: 1.2553 - val_mae: 0.8057 - lr: 1.5024e-04 - 2s/epoch - 3ms/step
Epoch 2/500
556/556 - 1s - loss: 20.3370 - mse: 20.3370 - mae: 2.1956 - val_loss: 1.7353 - val_mse: 1.7353 - val_mae: 1.0294 - lr: 1.5024e-04 - 969ms/epoch - 2ms/step
Epoch 3/500
556/556 - 1s - loss: 17.7971 - mse: 17.7971 - mae: 1.8429 - val_loss: 3.0400 - val_mse: 3.0400 - val_mae: 1.4513 - lr: 1.5024e-04 - 943ms/epoch - 2ms/step
Epoch 4/500
556/556 - 1s - loss: 15.6142 - mse: 15.6142 - mae: 1.5951 - val_loss: 5.3522 - val_mse: 5.3522 - val_mae: 2.0225 - lr: 1.5024e-04 - 950ms/epoch - 2ms/step
Epoch 5/500
556/556 - 1s - loss: 14.2710 - mse: 14.2710 - mae: 1.5684 - val_loss: 7.4026 - val_mse: 7.4026 - val_mae: 2.4252 - lr: 1.5024e-04 - 974ms/epoch - 2ms/step
Epoch 6/500
556/556 - 1s - loss: 13.6456 - mse: 13.6456 - mae: 1.6008 - val_loss: 8

[I 2022-10-10 10:23:43,054] Finished trial#48 resulted in value: 8.28. Current best value is 8.28 with parameters: {'activation': 'tanh', 'num_hidden_layer': 2, 'i': 4, 'learning_rate': 0.00015023638006021693}.


Score for inner fold : loss of 8.228475570678711
Trial Number:49
-----Fold:0--------
Epoch 1/500
556/556 - 2s - loss: 22.0492 - mse: 22.0492 - mae: 2.4264 - val_loss: 1.6078 - val_mse: 1.6078 - val_mae: 0.9514 - lr: 1.3763e-04 - 2s/epoch - 3ms/step
Epoch 2/500
556/556 - 1s - loss: 18.1436 - mse: 18.1436 - mae: 1.8609 - val_loss: 4.0701 - val_mse: 4.0701 - val_mae: 1.7227 - lr: 1.3763e-04 - 1s/epoch - 2ms/step
Epoch 3/500
556/556 - 1s - loss: 15.0814 - mse: 15.0814 - mae: 1.6118 - val_loss: 7.6401 - val_mse: 7.6401 - val_mae: 2.4816 - lr: 1.3763e-04 - 1s/epoch - 2ms/step
Epoch 4/500
556/556 - 1s - loss: 14.1778 - mse: 14.1778 - mae: 1.6329 - val_loss: 8.2020 - val_mse: 8.2020 - val_mae: 2.5516 - lr: 1.3763e-04 - 1s/epoch - 2ms/step
Epoch 5/500
556/556 - 1s - loss: 13.8127 - mse: 13.8127 - mae: 1.6141 - val_loss: 8.1829 - val_mse: 8.1829 - val_mae: 2.5344 - lr: 1.3763e-04 - 1s/epoch - 2ms/step
Epoch 6/500
556/556 - 1s - loss: 13.5624 - mse: 13.5624 - mae: 1.5967 - val_loss: 8.4920 - val_

[I 2022-10-10 10:24:07,493] Finished trial#49 resulted in value: 8.52. Current best value is 8.28 with parameters: {'activation': 'tanh', 'num_hidden_layer': 2, 'i': 4, 'learning_rate': 0.00015023638006021693}.


Score for inner fold : loss of 8.413633346557617
Number of finished trial : 50
Best Trial Value : 8.28


In [ ]:
2**6

64

In [ ]:
#model_retrain for outer cv (best model 1)

#  {'activation': 'tanh', 'num_hidden_layer': 2, 'i': 4, 'learning_rate': 0.00015023638006021693}
history_list_cv1=[]
optimizer = Adam(learning_rate=0.00015023638006021693,clipnorm=1.0)
model = create_model(activation="tanh", num_hidden_layer=2, num_hidden_unit=16)
model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
kfold = KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
es = EarlyStopping(monitor='mse',mode='min',patience=10)
for (train,test) in kfold.split(training,labelsForTrain):

  print('------------------------------------------------------------------------')
  print(f'Training for Outer fold {fold_no} ...')
  
  outer_cv_train=training.iloc[train]
  outer_cv_trainlabel=labelsForTrain.iloc[train]

  outer_cv_test=training.iloc[test]
  outer_cv_testlabel=labelsForTrain.iloc[test]

  reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=5, min_lr=0.001)
  # Fit data to model
  history = model.fit(outer_cv_train, outer_cv_trainlabel,
                  batch_size=64,
                  epochs=500,
                  verbose=2,
                  validation_data=(outer_cv_test,outer_cv_testlabel),
                  validation_batch_size=64,
                  callbacks=[es,reduce_lr])
  
  scores=model.evaluate(outer_cv_test,outer_cv_testlabel,verbose=0)
  print(scores)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1 
history_list_cv1.append(history)

------------------------------------------------------------------------
Training for Outer fold 1 ...
Epoch 1/500
834/834 - 2s - loss: 21.2012 - mse: 21.2012 - mae: 2.4027 - val_loss: 48.1052 - val_mse: 48.1052 - val_mae: 2.1927 - lr: 1.5024e-04 - 2s/epoch - 3ms/step
Epoch 2/500
834/834 - 1s - loss: 17.5816 - mse: 17.5816 - mae: 1.8639 - val_loss: 44.6163 - val_mse: 44.6163 - val_mae: 1.6841 - lr: 1.5024e-04 - 1s/epoch - 2ms/step
Epoch 3/500
834/834 - 1s - loss: 14.8383 - mse: 14.8383 - mae: 1.5634 - val_loss: 42.6120 - val_mse: 42.6120 - val_mae: 1.6119 - lr: 1.5024e-04 - 1s/epoch - 2ms/step
Epoch 4/500
834/834 - 1s - loss: 13.7971 - mse: 13.7971 - mae: 1.5958 - val_loss: 42.0510 - val_mse: 42.0510 - val_mae: 1.6412 - lr: 1.5024e-04 - 1s/epoch - 2ms/step
Epoch 5/500
834/834 - 1s - loss: 13.4445 - mse: 13.4445 - mae: 1.6026 - val_loss: 41.7488 - val_mse: 41.7488 - val_mae: 1.6392 - lr: 1.5024e-04 - 1s/epoch - 2ms/step
Epoch 6/500
834/834 - 1s - loss: 13.2161 - mse: 13.2161 - mae: 1.59

In [ ]:
results = model.evaluate(testing, labelsForTest, batch_size=10)

2000/2000 [==============================] - 3s 2ms/step - loss: 9.0166 - mse: 9.0166 - mae: 1.4660


In [ ]:
#model_retrain for outer cv (best model 2), similar with model1

# {'activation': 'tanh', 'num_hidden_layer': 2, 'i': 4, 'learning_rate': 0.00010020536598530568}
history_list_cv2=[]
optimizer = Adam(learning_rate=0.00010020536598530568,clipnorm=1.0)
model2 = create_model(activation="tanh", num_hidden_layer=2, num_hidden_unit=16)
model2.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
kfold = KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
es = EarlyStopping(monitor='mse',mode='min',patience=10)
for (train,test) in kfold.split(training,labelsForTrain):

  print('------------------------------------------------------------------------')
  print(f'Training for Outer fold {fold_no} ...')
  
  outer_cv_train=training.iloc[train]
  outer_cv_trainlabel=labelsForTrain.iloc[train]

  outer_cv_test=training.iloc[test]
  outer_cv_testlabel=labelsForTrain.iloc[test]

  reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=5, min_lr=0.001)
  # Fit data to model
  history = model2.fit(outer_cv_train, outer_cv_trainlabel,
                  batch_size=64,
                  epochs=500,
                  verbose=2,
                  validation_data=(outer_cv_test,outer_cv_testlabel),
                  validation_batch_size=64,
                  callbacks=[es,reduce_lr])
  
  scores=model2.evaluate(outer_cv_test,outer_cv_testlabel,verbose=0)
  print(scores)
  print(f'Score for fold {fold_no}: {model2.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1 
history_list_cv2.append(history)

------------------------------------------------------------------------
Training for Outer fold 1 ...
Epoch 1/1000
4000/4000 - 11s - loss: 21.5003 - mse: 21.5003 - mae: 1.5880 - 11s/epoch - 3ms/step
Epoch 2/1000
4000/4000 - 10s - loss: 20.4470 - mse: 20.4470 - mae: 1.5312 - 10s/epoch - 3ms/step
Epoch 3/1000
4000/4000 - 10s - loss: 20.1788 - mse: 20.1788 - mae: 1.5167 - 10s/epoch - 2ms/step
Epoch 4/1000
4000/4000 - 10s - loss: 19.9602 - mse: 19.9602 - mae: 1.5045 - 10s/epoch - 2ms/step
Epoch 5/1000
4000/4000 - 10s - loss: 19.8158 - mse: 19.8158 - mae: 1.4948 - 10s/epoch - 3ms/step
Epoch 6/1000
4000/4000 - 10s - loss: 19.6828 - mse: 19.6828 - mae: 1.4866 - 10s/epoch - 2ms/step
Epoch 7/1000
4000/4000 - 10s - loss: 19.6309 - mse: 19.6309 - mae: 1.4812 - 10s/epoch - 2ms/step
Epoch 8/1000
4000/4000 - 10s - loss: 19.6343 - mse: 19.6343 - mae: 1.4812 - 10s/epoch - 2ms/step
Epoch 9/1000


KeyboardInterrupt: ignored

In [ ]:
results = model_2.evaluate(testing, labelsForTest, batch_size=10)

In [ ]:
#model_retrain for outer cv (best model 3)
#{'activation': 'tanh', 'num_hidden_layer': 4, 'i': 3, 'learning_rate': 0.00010292007837467995}
history_list_cv3=[]
optimizer = Adam(learning_rate=0.00010292007837467995,clipnorm=1.0)
model3 = create_model(activation="tanh", num_hidden_layer=4, num_hidden_unit=8)
model3.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
kfold = KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
es = EarlyStopping(monitor='mse',mode='min',patience=10)
for (train,test) in kfold.split(training,labelsForTrain):

  print('------------------------------------------------------------------------')
  print(f'Training for Outer fold {fold_no} ...')
  
  outer_cv_train=training.iloc[train]
  outer_cv_trainlabel=labelsForTrain.iloc[train]

  outer_cv_test=training.iloc[test]
  outer_cv_testlabel=labelsForTrain.iloc[test]

  reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=5, min_lr=0.001)
  # Fit data to model
  history = model3.fit(outer_cv_train, outer_cv_trainlabel,
                  batch_size=64,
                  epochs=500,
                  verbose=2,
                  validation_data=(outer_cv_test,outer_cv_testlabel),
                  validation_batch_size=64,
                  callbacks=[es,reduce_lr])
  
  scores=model3.evaluate(outer_cv_test,outer_cv_testlabel,verbose=0)
  print(scores)
  print(f'Score for fold {fold_no}: {model3.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1 
history_list_cv3.append(history)

------------------------------------------------------------------------
Training for Outer fold 1 ...
Epoch 1/500
834/834 - 2s - loss: 36.4972 - mse: 36.4972 - mae: 2.3267 - val_loss: 17.4396 - val_mse: 17.4396 - val_mae: 1.8788 - lr: 1.0292e-04 - 2s/epoch - 3ms/step
Epoch 2/500
834/834 - 2s - loss: 31.9206 - mse: 31.9206 - mae: 1.7468 - val_loss: 14.1429 - val_mse: 14.1429 - val_mae: 1.6416 - lr: 1.0292e-04 - 2s/epoch - 2ms/step
Epoch 3/500
834/834 - 2s - loss: 29.9957 - mse: 29.9957 - mae: 1.7306 - val_loss: 13.5027 - val_mse: 13.5027 - val_mae: 1.6827 - lr: 1.0292e-04 - 2s/epoch - 2ms/step
Epoch 4/500
834/834 - 2s - loss: 29.5711 - mse: 29.5711 - mae: 1.7270 - val_loss: 13.2361 - val_mse: 13.2361 - val_mae: 1.6572 - lr: 1.0292e-04 - 2s/epoch - 2ms/step
Epoch 5/500
834/834 - 2s - loss: 29.3235 - mse: 29.3235 - mae: 1.7041 - val_loss: 13.0107 - val_mse: 13.0107 - val_mae: 1.6428 - lr: 1.0292e-04 - 2s/epoch - 2ms/step
Epoch 6/500
834/834 - 2s - loss: 29.1044 - mse: 29.1044 - mae: 1.68

In [ ]:
results = model3.evaluate(testing, labelsForTest, batch_size=10)

2000/2000 [==============================] - 3s 2ms/step - loss: 8.8159 - mse: 8.8159 - mae: 1.4581


In [ ]:
training

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.594081,0.904101,0.998045,-0.004595,0.394529,0.926423,0.880260,-0.053977,1.976635,0.173799,0.402216,0.152305,-0.299098,-0.407369,0.617311,-0.643731,-0.239689,1.452094
1,1.219960,-1.688930,-1.491378,2.535186,-0.172440,-0.366368,0.423633,-0.190553,-0.611129,-0.194835,0.814666,-0.181531,0.329854,10.726693,0.617311,1.475200,-1.087631,-0.473122
2,0.395851,0.255843,0.064511,-0.482978,0.319514,0.664912,0.609103,0.391782,1.976635,0.327396,-0.107280,0.264463,0.958806,-0.056630,-1.818900,-1.382147,1.037042,0.823739
3,-0.555657,0.255843,0.064511,-0.169854,-0.219542,0.011135,0.976132,-0.007639,0.359283,-0.141076,-0.374160,-0.066294,0.958806,-0.483286,0.617311,-0.386891,-0.086498,-0.218550
4,0.584975,-1.688930,-1.491378,0.795610,-0.306768,-0.400111,-0.386756,-0.190553,-0.611129,-0.194835,-3.297700,-0.182851,-0.928051,0.087614,0.617311,0.126789,-1.863465,-0.920809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,0.133394,0.904101,0.998045,-0.761310,-0.327702,-0.201869,0.646128,0.111351,-0.287658,-0.164116,1.154330,-0.154921,0.958806,0.599298,-0.600795,-0.980834,1.641533,1.645053
79996,-0.152830,-1.688930,-1.491378,-0.900476,0.089238,-0.389566,-1.231812,-0.190553,-0.611129,-0.194835,-0.058757,-0.182851,0.329854,1.232450,0.617311,0.608365,-1.519767,-0.329103
79997,-0.126048,0.904101,0.998045,0.317227,-0.006711,0.517285,0.463527,0.125043,1.653164,-0.018198,-0.046626,0.063678,0.958806,-0.223648,0.617311,-0.563469,0.257628,0.543124
79998,0.418950,-1.040672,-1.180200,1.273993,-0.154995,-0.113293,-0.744071,0.018797,0.035812,-0.033558,-0.471207,-0.097962,-0.299098,4.288888,0.617311,0.126789,-0.882083,0.210619


In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
#15.1
study.optimize(objective, n_trials=50, )

Epoch 1/10
4000/4000 - 5s - loss: 22.9177 - mse: 22.9177 - mae: 1.6737 - lr: 0.0024 - 5s/epoch - 1ms/step
Epoch 2/10


KeyboardInterrupt: ignored